In [ ]:
import jtrace
import re
import numpy as np
from ipywidgets import interact
import ipywidgets as widgets
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
hsc = """
#
# Column 0: Name
# Column 1: Type
# Column 2: Curvature R (mm)
# Column 3: Thickness dz (mm)
# Column 4: Outer Radius (mm)
# Column 5: Inner Radius (mm)
# Column 6: Conic Constant Kappa
# Column 7 - 20: Aspheric Coefficient a_3 - a_16 (a_n r^n in meters)
# Column 15: Coating file
# Column 16: Medium file
# (0)    (1)               (2)                (3)            (4) (5)      (6) (7) (8)                 (9) (10)                (11) (12)                (13) (14)                (15)                 (16) (17) (18)                (19) (20)                (21) (22)
PM    mirror 30000.0             0.0              4100.0         0.0 -1.00835 0.0 0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                  0.0  0.0                 0.0  0.0                 ---  air
G1    lens     760.0             13456.4          410.0          0.0 0.0      0.0 0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                  0.0  0.0                 0.0  0.0                 ---  silica
G1E   lens    1375.5             97.18            402.20143728   0.0 0.0      0.0 -1.497488783073E-10 0.0  -7.579068214673E-17 0.0  -7.705293869522E-22 0.0  1.026626319251E-26  0.0  -7.078826218129E-32  0.0  2.559600617806E-37  0.0  -3.761551480578E-43 ---  air
G2    lens   -3530.0             372.85           307.5181146466 0.0 0.0      0.0 6.914718060975E-11  0.0  6.03728126364E-16   0.0  -1.532623173462E-20 0.0  3.618110894954E-25  0.0  -4.741677881169E-30  0.0  3.217309071692E-35  0.0  -8.85987809163E-41  ---  bsl7y
G2E   lens   656.2498974113356   46.0             286.5165285897 0.0 0.0      0.0 0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                  0.0  0.0                 0.0  0.0                 ---  air
ADC1  lens   0.0                 318.0            282.794660157  0.0 0.0      0.0 0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                  0.0  0.0                 0.0  0.0                 ---  bsl7y
ADC1E lens   1058.0              40.0             282.0282677958 0.0 0.0      0.0 0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                  0.0  0.0                 0.0  0.0                 ---  air
ADC2  lens   1040.0              3.0              282.5217682803 0.0 0.0      0.0 0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                  0.0  0.0                 0.0  0.0                 ---  pbl1y
ADC2E lens   0.0                 82.0             281.8065578377 0.0 0.0      0.0 0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                  0.0  0.0                 0.0  0.0                 ---  air
G3    lens   -840.000170144847   274.3            275.8409560694 0.0 0.0      0.0 2.768476102307E-9   0.0  -4.855642009259E-14 0.0  7.176113272766E-19  0.0  -1.076370261452E-23 0.0  1.18744321781E-28    0.0  -7.983821197395E-34 0.0  2.393556961408E-39  ---  pbl1y
G3E   lens   9800.0              40.0             283.8358237172 0.0 0.0      0.0 0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                  0.0  0.0                 0.0  0.0                 ---  air
G4    lens   480.0               90.0             314.0          0.0 0.0      0.0 -4.355533915492E-9  0.0  3.635867914913E-14  0.0  -5.951267151922E-19 0.0  7.658839741989E-24  0.0  -7.194120343205E-29  0.0  3.942820924898E-34  0.0  -9.54342843455E-40  ---  bsl7y
G4E   lens   4021.7590211374127  102.0            314.0          0.0 0.0      0.0 0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                  0.0  0.0                 0.0  0.0                 ---  air
G5    lens   4176.748363640779   100.2            314.0          0.0 0.0      0.0 0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                  0.0  0.0                 0.0  0.0                 ---  silica
G5E   lens   -1272.8222984309575 88.0             308.0          0.0 0.0      0.0 -1.064687616564E-9  0.0  3.377750239208E-15  0.0  -1.1026471335E-19   0.0  2.282368582236E-24  0.0  -2.743039658171E-29  0.0  1.755771152898E-34  0.0  -4.821949608578E-40 ---  air
F     filter 0.0                 90.04125645151   269.5068874933 0.0 0.0      0.0 0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                  0.0  0.0                 0.0  0.0                 ---  silica
FE    filter 0.0                 15.0             266.8394944827 0.0 0.0      0.0 0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                  0.0  0.0                 0.0  0.0                 ---  air
W     lens   0.0                 32.5             258.2205358629 0.0 0.0      0.0 0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                  0.0  0.0                 0.0  0.0                 ---  silica
WE    lens   0.0                 37.0             251.6409664366 0.0 0.0      0.0 0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                  0.0  0.0                 0.0  0.0                 ---  air
D     det    0.0                 15.0             247.6837392465 0.0 0.0      0.0 0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                 0.0  0.0                  0.0  0.0                 0.0  0.0                 ---  ---
"""
re.sub("\s+"," ", hsc)
None

In [ ]:
bsl7y = jtrace.ConstMedium(1.50939738)
silica = jtrace.ConstMedium(1.45172729)
pbl1y = jtrace.ConstMedium(1.53789058)
air = jtrace.ConstMedium(1.0)

In [ ]:
def rays(theta_x, theta_y, wavelength):
    # Point towards (0,0,0), but at an angle.  Need to determine pupil locations.
    rs = np.linspace(telescope[0]['inner'], telescope[0]['outer'], 40)
    # The above works if theta is 0.
    # If theta is not zero, then need to shift the rays depending on how far away their origins are.
    # We'll set the z-origin of the rays to be 25 meters above the M1 vertex.
    z = 25
    dx = z * np.tan(theta_x)
    dy = z * np.tan(theta_y)
    rays_ = []
    for r in rs:        
        phis = np.linspace(0, 2*np.pi, int(256*r/telescope[0]['outer']), endpoint=False)
        for phi in phis:
            rays_.append(
                jtrace.Ray(jtrace.Vec3(r*np.cos(phi)+dx, r*np.sin(phi)+dy, z),
                           jtrace.Vec3(-np.tan(theta_x), -np.tan(theta_y), -1).UnitVec3()/air.getN(wavelength),
                           0, wavelength))
    return rays_

In [ ]:
def traceMany(rays, telescope):
    rs = jtrace.RayVector(rays)
    for optic in telescope:
        isecs = optic['surface'].intersect(rs)
        if optic['typ'] == 'mirror':
            rs = jtrace._jtrace.reflectMany(isecs, rs)
        elif optic['typ'] in ['lens', 'filter']:
            rs = jtrace._jtrace.refractMany(isecs, rs, optic['m0'], optic['m1'])
    return rs, isecs

def perturb(PM_dx=0, PM_dy=0, 
            cam_dx=0, cam_dy=0, cam_dz=0):
    telescope = []
    z = 0.0
    m0 = air
    m1 = silica
    for line in hsc.split('\n'):
        if len(line) == 0 : continue
        if line[0] == '#': continue
        name, typ, R, dz, outer, inner, kappa, a3, a4, a5, a6, a7, a8, a9, a10, a11, a12, a13, a14, a15, a16, coating, medium = line.split()
        z += float(dz)/1000
        if typ == 'none': continue
        m0 = m1
        if medium == 'air': 
            m1 = air
        elif medium == 'pbl1y':
            m1 = pbl1y
        elif medium == 'bsl7y':
            m1 = bsl7y
        elif medium == 'silica':
            m1 = silica
        elif medium == '---':
            pass
        else:
            raise ValueError("Unknown medium {}".format(medium))

        if float(R) == 0:
            surface = jtrace.Plane(z, float(inner)/1000, float(outer)/1000)
        else:
            # Notice the manipulation of the raw aspheric coefficients below.  There's a negative sign missing, 
            # and also the coefficients are for mm instead of m, so there are factors of 1000^4, ^6, and ^8 
            # missing as well.  Actually, the factors or 1000^(4-1), ^(6-1), ^(8-1) for some weird reason.  It's 
            # like the zemax crowd just changed the coefficients from m to mm by multiplying by 1000 instead of
            # 1000 to the appropriate power.  Weird.
            alpha = [np.double(a)*10**(3*(i-1)) 
                     for a, i in zip([a4, a6, a8, a10, a12, a14, a16],
                                     [4, 6, 8, 10, 12, 14, 16])]
            if all(a == 0.0 for a in alpha):
                surface = jtrace.Quadric(float(R)/1000, float(kappa), z, 
                                         float(inner)/1000, float(outer)/1000)
            else:
                surface = jtrace.Asphere(float(R)/1000, float(kappa), alpha, z,
                                         float(inner)/1000, float(outer)/1000)
        if name == 'PM':
            surface = surface.shift(PM_dx, PM_dy, 0)
        else:
            surface = surface.shift(cam_dx, cam_dy, cam_dz)
        telescope.append(dict(name=name, surface=surface, 
                         outer=float(outer)/1000, inner=float(inner)/1000, 
                         m0=m0, m1=m1, typ=typ))
    return telescope
telescope = perturb()

In [ ]:
@interact(wavelen=widgets.FloatSlider(min=300.0,max=1200.0,step=10.0,value=620.0),
          theta_x=widgets.FloatSlider(min=-0.75,max=0.75,step=0.01,value=-0.40),
          theta_y=widgets.FloatSlider(min=-0.75,max=0.75,step=0.01,value=0.0),
          PM_dx=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.00),
          PM_dy=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.0),
          cam_dx=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.0),
          cam_dy=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.0),
          cam_dz=widgets.FloatSlider(min=-10, max=10, step=0.01, value=0.0),
          logscale=widgets.FloatSlider(min=1.2, max=3, step=0.1, value=1.2))
def spot(wavelen, theta_x, theta_y, PM_dx, PM_dy, cam_dx, cam_dy, cam_dz, logscale):
    """Display a spot diagram for LSST.

    @param wavelen  Wavelength in nm
    @param theta_x  Field angle in degrees
    @param theta_y  Field angle in degrees
    @param PM_dx    Primary mirror x decenter in mm
    @param PM_dy    Primary mirror y decenter 
    @param cam_dx   Camera x decenter in mm
    @param cam_dy   Camera y decenter in mm
    @param cam_dz   Camera z despace in mm
    @param logscale Logarithmic axes zoom level
    """
    telescope = perturb(PM_dx*1e-3, PM_dy*1e-3, 
                        cam_dx*1e-3, cam_dy*1e-3, cam_dz*1e-3)
    rs, isecs = traceMany(rays(theta_x*np.pi/180, theta_y*np.pi/180, wavelen/1000), telescope)
    spots = [(isec.x0, isec.y0) for isec, ray in zip(isecs, rs) if not ray.isVignetted]
    spots = np.array(spots)
    spots -= np.mean(spots, axis=0)
    spots *= 1e6 # meters -> microns
    print(np.mean(spots, axis=0), np.std(spots, axis=0))
    plt.figure(figsize=(4.5,4))
    plt.scatter(spots[:,0], spots[:,1], s=1, alpha=0.2)
    plt.xlim(-10**logscale, 10**logscale)
    plt.ylim(-10**logscale, 10**logscale)
    plt.title(r"$\theta_x = {:4.2f}\,,\theta_y = {:4.2f}$".format(theta_x, theta_y))
    plt.xlabel("microns")
    plt.ylabel("microns")

In [ ]:
#  http://stackoverflow.com/a/18968498
def planeFit(points):
    """
    p, n = planeFit(points)

    Given an array, points, of shape (d,...)
    representing points in d-dimensional space,
    fit an d-dimensional plane to the points.
    Return a point, p, on the plane (the point-cloud centroid),
    and the normal, n.
    """
    from numpy.linalg import svd
    points = np.reshape(points, (np.shape(points)[0], -1)) # Collapse trialing dimensions
    ctr = points.mean(axis=1)
    x = points - ctr[:,np.newaxis]
    M = np.dot(x, x.T) # Could also use np.cov(x) here.
    return ctr, svd(M)[0][:,-1]

In [ ]:
@interact(wavelen=widgets.FloatSlider(min=300.0,max=1200.0,step=10.0,value=620.0),
          theta_x=widgets.FloatSlider(min=-0.75,max=0.75,step=0.01,value=-0.40),
          theta_y=widgets.FloatSlider(min=-0.75,max=0.75,step=0.01,value=0.0),
          PM_dx=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.00),
          PM_dy=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.0),
          cam_dx=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.0),
          cam_dy=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.0),
          cam_dz=widgets.FloatSlider(min=-10, max=10, step=0.01, value=0.0),
          logscale=widgets.FloatSlider(min=-6, max=-4, step=0.1, value=-5.5))
def opd(wavelen, theta_x, theta_y, PM_dx, PM_dy, cam_dx, cam_dy, cam_dz, logscale):
    """Display optical path differences

    @param wavelen  Wavelength in nm
    @param theta_x  Field angle in degrees
    @param theta_y  Field angle in degrees
    @param PM_dx    Primary mirror x decenter in mm
    @param PM_dy    Primary mirror y decenter in mm
    @param cam_dx   Camera x decenter in mm
    @param cam_dy   Camera y decenter in mm
    @param cam_dz   Camera z despace in mm
    @param logscale Logarithmic colorbar zoom level
    """
    telescope = perturb(PM_dx*1e-3, PM_dy*1e-3, 
                        cam_dx*1e-3, cam_dy*1e-3, cam_dz*1e-3)
    rs, isecs = traceMany(rays(theta_x*np.pi/180, theta_y*np.pi/180, wavelen), telescope)
    theta_opd = [(r.v.x, r.v.y, isec.t) for r, isec in zip(rs, isecs) if not r.isVignetted]
    theta_opd = np.array(theta_opd)
    opd = theta_opd[:,2]
    opd[:] -= np.mean(opd)
    x = theta_opd[:,0]
    y = theta_opd[:,1]
    p, n = planeFit(theta_opd[::10,:].T)
    const = np.dot(p, n)
    opd[:] -= (const-n[0]*x-n[1]*y)/n[2]
    plt.figure(figsize=(5.3,4))
    plt.scatter(x, y, c=opd, s=5, vmin=-10**logscale, vmax=10**logscale)
    plt.xlim(-0.4, 0.4)
    plt.ylim(-0.4, 0.4)
    plt.axhline(0.0, c='k')
    plt.axvline(0.0, c='k')
    plt.xlabel("vx")
    plt.ylabel("vy")
    plt.colorbar()